# Display and record via HOZAN USB camera

## 1) Test for camera

In [5]:
# Test for the availability of camera
import cv2
import time

SRvideo = cv2.VideoCapture(1)
SRvideo.set(cv2.CAP_PROP_FPS,30)
# the fps of the camera is between 1 to 40, which can be configured.
if SRvideo.isOpened():
    print('The camera is available')
    print(SRvideo.get(5))

SRvideo.release()

The camera is available
30.0


In [2]:
# Opend and display via USB camera
import cv2

# read from the camera via creating a VideoCapture object
SRvideo = cv2.VideoCapture(1)
if not SRvideo.isOpened(): 
    print("Cannot read the camera")

while SRvideo.isOpened():
    ret, frame = SRvideo.read()
    if ret == True:    
        # Display the resulting frame  
        frame = cv2.rotate(frame, cv2.ROTATE_180)
        cv2.imshow('SRcrawling',frame)
        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  

# Finally, release the capture and write object, then close windows
SRvideo.release()
cv2.destroyAllWindows() 

In [1]:
# open the USB camera, show and record the video frame
import cv2
import numpy as np

# read from the camera via creating a VideoCapture object
SRvideo = cv2.VideoCapture(1)
if not SRvideo.isOpened(): 
    print("Cannot read the camera")

# save the video with the configuration of video encode format, fps, frame size,
frame_width = int(SRvideo.get(3))
frame_height = int(SRvideo.get(4))
SRvideo.set(cv2.CAP_PROP_FPS,40)
# shift the width and height considering the camera rotation
SRwri = cv2.VideoWriter('SRdeformationTest.avi',cv2.VideoWriter_fourcc(*'MJPG'), 40, (frame_width,frame_height))

while SRvideo.isOpened():
    ret, frame = SRvideo.read()
    if ret == True:  
        # rotate the frame, display and video save
        frame = cv2.rotate(frame, cv2.ROTATE_180)
        cv2.imshow('SRcrawling',frame)
        SRwri.write(frame)
        # Press Q on keyboard to stop recording
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break  

# Finally, release the capture and write object, then close windows
SRvideo.release()
SRwri.release()
cv2.destroyAllWindows() 

## 2) Display video in GUI window

In [4]:
# Integration in a whole class
import cv2
import tkinter
from tkinter import ttk
import PIL.Image, PIL.ImageTk
import time

class Application:
    def __init__(self, window, window_title, video_source):
        # USB camera source
        self.SRvideo = cv2.VideoCapture(video_source)
        if self.SRvideo.isOpened():
            print('camera is available')
        # the size of HOZAN USB camera is 640*480
        self.width = int(self.SRvideo.get(cv2.CAP_PROP_FRAME_WIDTH))   # cv2.CAP_PROP_FRAME_WIDTH = 3
        self.height = int(self.SRvideo.get(cv2.CAP_PROP_FRAME_HEIGHT)) # cv2.CAP_PROP_FRAME_HEIGHT = 4
        self.SRvideo.set(cv2.CAP_PROP_FPS, 40)  # cv2.CAP_PROP_FPD = 5
        print(self.SRvideo.get(5))
        # shift the height and width considering the camera rotation        
        self.SRwri = cv2.VideoWriter('SRcrawling.avi', cv2.VideoWriter_fourcc(*'MJPG'), 40, (self.width, self.height))
        
        # configure the GUI window
        self.window = window
        self.window.title(window_title)
        self.window.geometry('1280x480')
        # Canvas configuration: inlucding video frame, button and pressure sensor curves
        self.canvas = tkinter.Canvas(window, width = self.width, height = self.height)
        self.canvas.pack(side=tkinter.LEFT)
        # using new button format, it would be better when we use a add a ttk.frame widget before.
        self.btn_snapshot = ttk.Button(window, text = 'Snapshot', width = 50, command = self.snapshot)
        self.btn_snapshot.pack(anchor = tkinter.CENTER, expand = True)
        self.btn_stop = ttk.Button(window, text = 'Stop',width = 50, command = self.stop)
        self.btn_stop.pack(anchor = tkinter.CENTER, expand = True)
        
        # Updata the frame every 10 milliseconds
        self.delay = 10
        self.update()
        self.window.mainloop()
        
    # function for the frame update
    def update(self):
        self.ret, self.frame = self.SRvideo.read()
        self.frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        self.frame = cv2.rotate(self.frame, cv2.ROTATE_180)
        if self.ret:
            self.photo = PIL.ImageTk.PhotoImage(PIL.Image.fromarray(self.frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)
        self.window.after(self.delay, self.update)
        self.SRwri.write(cv2.cvtColor(self.frame, cv2.COLOR_RGB2BGR))
        
    # function for snapshot 
    def snapshot(self):
        self.ret, self.frame = self.SRvideo.read()
        self.frame = cv2.cvtColor(self.frame, cv2.COLOR_BGR2RGB)
        self.frame = cv2.rotate(self.frame, cv2.ROTATE_180)
        if self.ret:
            cv2.imwrite("SRcrawling-" + time.strftime("%Y-%m-%d-%H-%M-%S") + ".jpg", cv2.cvtColor(self.frame, cv2.COLOR_RGB2BGR))
    
    def stop(self):
        self.SRvideo.release()
        self.SRwri.release()
        cv2.destroyAllWindows() 
            
# create a window and pass it to the Application object
Application(tkinter.Tk(), "SRcrawling", 1)

camera is available
40.0


In [ ]:
# use different class to express different function
import tkinter
import cv2
import PIL.Image, PIL.ImageTk
import time


# main class for the GUI display
class Application:
    def __init__(self, window, window_title, video_source=1):
        # open the HOZAN USB camera(1, the computer camera is 0)
        self.SRvid = SRVideoCapture(video_source)
        
        # configure the GUI window
        self.window = window
        self.window.title(window_title)
#         self.window.geometry('1280*550')
        # Create a canvas that can fit the above video source size
        self.canvas = tkinter.Canvas(window, width = self.SRvid.width, height = self.SRvid.height)         
        self.canvas.pack()     
        # Button that lets the user take a snapshot
        self.btn_snapshot=tkinter.Button(window, text="Snapshot", width=50, command=self.snapshot)
        self.btn_snapshot.pack(anchor=tkinter.CENTER, expand=True)

        # After it is called once, the update method will be automatically called every delay milliseconds
        self.delay = 50
        self.update()
        self.window.mainloop()
    
    # update GUI window
    def update(self):
        # Get a frame from the video source
        ret, frame = self.SRvid.get_frame()
        if ret:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(frame))
            self.canvas.create_image(0, 0, image = self.photo, anchor = tkinter.NW)
        self.SRvid.SRwri.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        self.window.after(self.delay, self.update)
    # snapshot during locomotion
    def snapshot(self):
        # Get a frame from the video source
        ret, frame = self.SRvid.get_frame()
        if ret:
            cv2.imwrite("SRcrawling-" + time.strftime("%d-%m-%Y-%H-%M-%S") + ".jpg", cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

            
# Video management class
class SRVideoCapture:
    def __init__(self, video_source=1):
        self.SRvid = cv2.VideoCapture(video_source)
        if not self.SRvid.isOpened():
            print('Cannot read the camera')

        # the size of HOZAN USB camera is 640*480
        self.width = int(self.SRvid.get(3))   # cv2.CAP_PROP_FRAME_WIDTH = 3
        self.height = int(self.SRvid.get(4)) # cv2.CAP_PROP_FRAME_HEIGHT = 4
        # shift the height and width considering the camera rotation        
        self.SRwri = cv2.VideoWriter('SRcrawling.avi', cv2.VideoWriter_fourcc(*'MJPG'), 20, (self.width, self.height))
    
    # frame configuration
    def get_frame(self):
        ret, frame = self.SRvid.read()
        frame = cv2.rotate(frame, cv2.ROTATE_180)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        if ret:
            return (ret, frame)
        else:
            return (ret, None)

    # Release the video source when the object is destroyed
    def __del__(self):
        if self.SRvid.isOpened():
            self.SRvid.release()
            self.SRwri.release()

# Create a window and pass it to the Application object
Application(tkinter.Tk(), "SRcrawling")